## Problem Description

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?



In [267]:
import re
import numpy as np

In [1]:
# Read each row of the input file into a list of strings
with open('input1.txt') as f:
    content = f.readlines()

content = [x.strip() for x in content]
content

['.......497...........................858...923...128..................227..801........487.....664...........................................',
 '436........765..............140.......+....................859.............*.........+.................960........668.......................',
 '...*982...........=..........=....203......266.263...375*....=...402....691..-....................*..........575....................13......',
 '.............114...588...............*............*......631........*.......952...463..14.......661..........=...706......*333.........595..',
 '...194.........*..............743...917.&......375.....................................*...............544*.......*....664..................',
 '...*.....807..452....81..........*......969..#......309*................/....873....941...828.197..........427.728...............566...13...',
 '.243........*.....80.......329....470.......145.475.....111........*659..259....+........*....%........569..............%.....*.

In [269]:
def account_for_edges(idx: int, loc_min: int, loc_max: int):
    return min(max(idx, loc_min), loc_max)

def get_peripherals(start_x: int, end_x: int, loc_y: int, X: list[list]) -> list:
    # Get the peripheral chars of a number with start_x(inclusive) and end_x(non inclusive) location,
    #  and loc_y of X matrix to a tuple list
    dirs = [-1, 0, 1]
    ys = [account_for_edges(loc_y+dir, 0, len(X)-1) for dir in dirs]
    xs = list(range(account_for_edges(start_x+dirs[0], 0, len(X[0])), account_for_edges(end_x+dirs[-1], 0, len(X[0]))))
    combos = set([(y, x) for y in ys for x in xs])

    # Account for xs, ys pointing to entry
    number_tuples = set([(loc_y, x) for x in range(start_x, end_x)])
    chars = [X[y][x] for y,x in list(combos - number_tuples)]
    return chars

In [270]:
# Testing get_peripherals:
get_peripherals(1,8,8,content)

['.',
 '.',
 '.',
 '*',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.',
 '.']

- Save input in nxm grid

- Get all numbers and check peripherals to see if there is symbol
- If there is then add to sum

NOTES:
1. No wrapping allowed, check for out of bounds conditions
2. Careful how you check for symbols to exclude '\.'


In [272]:
def find_numbers(X: list[list]):
    nums = []
    for y, line in enumerate(X):
        number_matches = re.finditer(r'(\d+)', line)
        for num in number_matches:
            nums.append((int(num.group()), (num.span(), y)))

    return nums

In [275]:
sum = 0
for num in find_numbers(content):
    value = num[0]
    xs, y = num[1]
    chars = get_peripherals(*xs, y, content)

    # Find way to check for symbols
    for char in set(chars):
        pattern = re.compile(r'(\w|\.)')
        found = re.match(pattern, char)
        
        if found is None:
            sum += value


In [276]:
sum

540131